In [1]:
require "hdf5"
require "optim"
require "nn"



In [2]:
f = hdf5.open("data.hdf5", "r")

X_train = f:read("X_train"):all()
Y_train = f:read("Y_train"):all()
X_valid = f:read("X_valid"):all()
Y_valid = f:read("Y_valid"):all()
X_test = f:read("X_test"):all()
nwords = f:read("nwords"):all()[1]
nclasses = f:read("nclasses"):all()[1]


--sentences
X_valid_sen = f:read("X_valid_sen"):all()
X_test_sen = f:read("X_test_sen"):all()

--MEMM
X_train_MEMM = f:read("X_train_MEMM"):all()
X_valid_MEMM = f:read("X_valid_MEMM"):all()
X_valid_sen_MEMM = f:read("X_valid_sen_MEMM"):all()
nfeaturesMEMM = f:read("nfeaturesMEMM"):all()[1]

C = nclasses

In [95]:
function process(inputs, numwords)
    processed = inputs:clone()
    for i = 1, inputs:size(1) do
        processed[i][2] = inputs[i][2] + numwords
    end
    return processed
end

X_ = process(X_train_MEMM, nwords)
y = Y_train

In [88]:



addB = nn.Add(nclasses)
lookup = nn.LookupTable(nwords+nclasses,nclasses)
sum = nn.Sum(2)
softmax = nn.LogSoftMax()
h = nn.Sequential()
h:add(lookup)
h:add(sum)
h:add(addB)
mse = nn.CrossEntropyCriterion()

w, dl_dw = h:getParameters()

function predict(input, index1, index2, network)
    logscore = torch.log(nn.SoftMax():forward(h:forward(input:sub(index1,index2))))
    return logscore:expand(C,C)
end

In [6]:
function feval(w_new)
   
    bsize= batch
    local idx = torch.randperm(X_:size(1)):sub(1,bsize)

    local x = torch.Tensor(bsize, X_:size(2))

    
    --y = y:reshape(Y_train:size(1), 1)
    
    local y_ = torch.Tensor(bsize, 1)

    
    for i=1,bsize do
        x[i] = X_[idx[i]]
        
        y_[i] = y[idx[i]]
    end
    
    y_ = y_:squeeze()
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()
    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))
    
    return loss_w, dl_dw
            
end


In [103]:
batch = 1000

-- cycle on data
for i = 1,10000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,w)

    if i % 100 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

loss for iteration 100 is 0.22786392498671	


loss for iteration 200 is 0.20862948785873	


loss for iteration 300 is 0.1961132135018	


loss for iteration 400 is 0.20939169074304	


loss for iteration 500 is 0.20857774963998	


...

loss for iteration 9600 is 0.14444503020408	


loss for iteration 9700 is 0.15646697791733	


loss for iteration 9800 is 0.19698408166786	


loss for iteration 9900 is 0.1714771464837	


loss for iteration 10000 is 0.15240118407142	


In [115]:
function viterbi(observations, logscore)
    local initial = torch.zeros(nclasses,1) + .01
    initial[8] = 1.0
    initial = initial / torch.sum(initial)
    
    local n = observations:size(1)
    local max_table = torch.Tensor(n, C)
    local backpointer_table = torch.Tensor(n, C)
    -- first timestep
    -- the initial most likely paths are the initial state distribution
    -- NOTE: another unnecessary Tensor allocation here
    local maxes, backpointers = torch.log(initial):max(2)--(init+ emi[observations[1]]):max(2)
    max_table[1] = maxes
    -- remaining timesteps ("forwarding" the maxes)
    for i=2,n do
        -- precompute edge scores
        
        pred = predict(observations, i, i, logscore)
        scores = pred + maxes:view(1, C):expand(C, C)
        -- compute new maxes (NOTE: another unnecessary Tensor allocation here)
        maxes, backpointers = scores:max(2)
        -- record
        max_table[i] = maxes
        backpointer_table[i] = backpointers
        end
    -- follow backpointers to recover max path
    local classes = torch.Tensor(n)
    maxes, classes[n] = maxes:max(1)
    for i=n,2,-1 do
        classes[i-1] = backpointer_table[{i, classes[i]}]
    end
    return classes:sub(1,-1)
end

function predict_tags(X)
    local predictions = torch.zeros(X:size(1), X:size(2))
    for i=1,X:size(1) do
        local sen = (X[i]:sub(1,torch.nonzero(X[{{i,i},{},{1,1}}]):size(1))):squeeze()
   
        local p = viterbi(sen, predict)
        
        predictions[{{i,i},{1,p:size(1)}}] = p
    end
    return predictions
end

In [116]:
a =X_valid_sen_MEMM
predictions = predict_tags(a)


In [117]:
z = predict_tags(a)

In [119]:
print(z[1])

 1
 8
 8
 4
 8
 8
 8
 8
 8
 8
 8
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 63]



In [113]:
Y_valid:sub(1,100)

 8
 1
 1
 4
 1
 1
 1
 1
 1
 1
 1
 1
 8
 3
 1
 8
 5
 5
 1
 2
 2
 1
 1
 1
 1
 1
 1
 1
 4
 1
 4
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 8
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 4
 1
 4
 1
 4
 1
 1
 1
 1
 1
 1
 4
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 4
 1
 8
 1
 1
 4
 1
 1
 1
 1
 1
 1
 1
[torch.LongTensor of size 100]

